# WEFE Site Analyst Tool
## implemented in Jupyter Notebook and applied for a Case Study in Portugal

This Jupyter Notebook collects environmental and socioeconomic data for the planning of integrated water, energy, food, and environment systems from open servers.

In [13]:
# imports
import os
import pandas as pd
import matplotlib.pyplot as plt
# import files from source
os.chdir("../../src/")
import era5
import ee_layer
os.chdir("../examples/Portugal")

In [10]:
# initiate earth engine
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code:  4/1AWgavdeBwNDp5xhNCorJCNHqNpQvbM41WeJ8AQksxO0ChT9E4meNVki7uhI



Successfully saved authorization token.


In [14]:
# provide case study name
name = 'permontanha'
# coordinates
lat = 37.134986
lon = -8.853894
# set center of maps and zoom level
center = [lat, lon]
zoom = 17

In [19]:
# create a folium elevation map
map = folium.Map(location=center, zoom_start=zoom)

# Load the shapefile as GeoJsnon into folium map/ better: create shapefile from points
folium.GeoJson("pp_geometry.shp", name = 'geojson').add_to(map) 
point = ee.Geometry.Point(lon, lat)
buffer = point.buffer(1000)

# Define the study area as a Geometry object
# study_area = ee.Geometry.Polygon(
#    [[[-122.44, 37.75],
#      [-122.44, 37.63],
#      [-122.35, 37.63],
#      [-122.35, 37.75]]])
# display the map
map

In [23]:
# Use the Digital Elevation Model (DEM) to get the elevation values for the study area
elevation = ee.Image("USGS/SRTMGL1_003").clip(buffer)

# Add the elevation layer to the map
map.add_ee_layer(elevation, {'min': 70, 'max': 150, 'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}, 'elevation')
# min max should be selected given the elevation range found in the polygon
# Add the shapefile to the map
# in case of problems; use geopandas to create geojson from shapefile and add generated geojson into folium

folium.GeoJson("pp_geometry.shp", name = 'geojson').add_to(map)

# display the map
map


In [19]:
# SRTM map

# Set the area of interest (AOI)
# You can use the ee.Geometry.Point function to define a point and the ee.Geometry.Buffer function to create a buffer around the point
point = ee.Geometry.Point(lon, lat)
buffer = point.buffer(3000)  # 3 km buffer

# Load the SRTM elevation data
srtm = ee.Image("USGS/SRTMGL1_003")

# Get the elevation at the map center
elevation = ee.Image('USGS/SRTMGL1_003').sample(point)

# Get the elevation value at the map center
elevation_value = elevation.get('elevation').getInfo()

# Print the elevation value
print(f'Elevation at {lon}, {lat}: {elevation_value} meters')

# Make pixels with elevation below sea level transparent.
elv_img = srtm.updateMask(srtm.gt(0))

# Display the thumbnail of styled elevation
from IPython.display import Image

Image(url=elv_img.getThumbURL({
    'min': 0, 'max': 4000, 'dimensions': 512, 'region': buffer,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}))


Elevation at -83.616358, 9.830153: None meters


In [ ]:
# set start and end date (end date will be included
# in the time period for which data is downloaded)
start_date, end_date = '2020-12-31', '2021-12-31'  # time in UTC choose start date one day before time of interest
# for position east of 0° meridian for covering all hours of interest

In [ ]:
# download era5 dataset

variable = "wefesiteanalyst"
target_file = 'era5_wefesiteanalyst_'+name+'.nc'
print(target_file)

ds = era5.get_era5_data_from_datespan_and_position(
    variable=variable,
    start_date=start_date, end_date=end_date,
    latitude=latitude, longitude=longitude,
    target_file=target_file)

In [ ]:
era5_wefe = pd.read_csv(r"wefe.csv")
era5_wefe.head()

In [ ]:
# plot global horizontal irradiance
import matplotlib.pyplot as plt

era5_wefe.loc[:, ['ghi']].plot(title='Irradiance')
plt.xlabel('Time')
plt.ylabel('Irradiance in $W/m^2$')